In [ ]:
from experiments.datasets import DATASETS
from distribution_optimization_py.problem import ScaledGaussianMixtureProblem
import random 
import numpy as np
from leap_ec.representation import Representation

from distribution_optimization_py.ga_style_operators import GAStyleEA
from leap_ec.problem import FunctionProblem
from pyhms.problem import EvalCutoffProblem

In [ ]:
dataset = DATASETS[1]
for seed in range(1, 31):
    random.seed(seed)
    np.random.seed(seed)
    problem = ScaledGaussianMixtureProblem(dataset.data, dataset.nr_of_modes)
    bounds = np.array([[0, 1]] * problem.lower.shape[0])

    function_problem = FunctionProblem(problem, maximize=False)
    n_evaluations = 10000
    pop_size = 50
    eval_cutoff_problem = EvalCutoffProblem(function_problem, n_evaluations)
    ea = GAStyleEA(
        generations=1,
        problem=eval_cutoff_problem,
        bounds=bounds,
        pop_size=pop_size,
        k_elites=5,
        p_mutation=0.8,
        p_crossover=0.9,
        representation=Representation(
            initialize=lambda: problem.initialize()
        ),
    )
    best_fitness = np.inf
    all_populations = []
    population = None
    for i in range(n_evaluations // pop_size):
        population = ea.run(population)
        all_populations.extend(population)
    print(f"Seed: {seed} Fitness: {max(all_populations).fitness}")